In [ ]:
! pip install gpxpy

In [ ]:
! pip install pykml

In [ ]:
import os

In [ ]:
# https://stackoverflow.com/questions/3844430/how-to-get-the-duration-of-a-video-in-python
import json
import subprocess

#===============================
def getMediaInfo(mediafile):
    cmd = "mediainfo --Output=JSON %s"%(mediafile)
    proc = subprocess.Popen(cmd, shell=True,stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, stderr = proc.communicate()
    data = json.loads(stdout)
    return data

#===============================
def getDuration(mediafile):
    data = getMediaInfo(mediafile)
    duration = float(data['media']['track'][0]['Duration'])
    return duration

#===============================
def getFrameRate(mediafile):
    data = getMediaInfo(mediafile)
    frameRate = float(data['media']['track'][0]['FrameRate'])
    return frameRate

#===============================
def getFrameCount(mediafile):
    data = getMediaInfo(mediafile)
    frameCount = float(data['media']['track'][0]['FrameCount'])
    return frameCount

## GPX file

In [ ]:
gps_path = '../gpssamples/'
#gpx_file = 'adour.gpx' 
gpx_file = '3145554.gpx'
gpx_path = os.path.join(gps_path,gpx_file)

In [ ]:
import gpxpy
import gpxpy.gpx 
gpx_file = open(gpx_path,'r',encoding='utf-8')
gpx = gpxpy.parse(gpx_file) # data from parsed gpx file

In [ ]:
''' gpsPointList function extract gps points from gpx file '''
''' gpxdata is a gpxpy object that returns data from a parsed gpx file'''
''' gpsPointList return a list of dictionnary points with Time, Lat, Long, Elev'''
def gpsPointList(gpxdata):
    point_list = []
    for track in gpxdata.tracks:
        for segment in track.segments: 
            for point in segment.points:
                point_info = {'Time':point.time,'Latitude':point.latitude,'Longitude':point.longitude,'Elevation':point.elevation}
                point_list.append(point_info)
    return point_list

In [ ]:
mypoints = gpsPointList(gpx)
for i in range(0,10):
    print(mypoints[i])

## KML File

In [ ]:
gps_path = '../kmlsamples'
kml_file4 = '19022020_Adour_4.kml'
kml_path4 = os.path.join(gps_path,kml_file4)
kml_file5 = '19022020_Adour_5.kml'
kml_path5 = os.path.join(gps_path,kml_file5)
kml_file6 = '19022020_Adour_6.kml'
kml_path6 = os.path.join(gps_path,kml_file6)

## KML processing first tests

In [ ]:
from pykml import parser
gps_path = '../gpssamples'
kml_file = '19022020_Adour_6.kml'
kml_path = os.path.join(gps_path,kml_file)
root = parser.fromstring(open(kml_path, 'r').read())
print (root.Document.Placemark.Point.coordinates)

In [ ]:
# Getting Placemark child info example
for placemark in root.Document.Placemark:
    print(placemark.name)
    print(placemark.styleUrl)

In [ ]:
# Placemark 0 example: getting Extended info
for placemark in root.Document.Placemark:
    if(placemark.name == 0):
        print("Placemark Name: ",placemark.name)
        test = placemark

for SimpleData in test.ExtendedData.SchemaData.SimpleData:
        print(SimpleData)

In [ ]:
''' placeMarkNameList function takes a parsed kml file input'''
''' then returns the a list of Placemark names'''
def placemarkNameList(kmlroot):
    placemark_name = []
    for placemark in kmlroot.Document.Placemark:
        placemark_name.append(placemark.name)
    return placemark_name

# Testing placemarkNameList function
mykmlPlacemarkList = placemarkNameList(root)
print(mykmlPlacemarkList)

In [ ]:
''' getPlacemarkExtendedData function takes a placemarkname list input '''
''' then return'''
def getPlacemarkExtendedData(placemarkname):
    placemarkExtendedData_list = []
    for i in range(1,len(placemarkname)-1):
        
        placemarkExtendedData_info = []
        for placemark in root.Document.Placemark:
            if(placemark.name == placemarkname[i]):
                #print(placemark.name)
                temp = placemark
        for SimpleData in temp.ExtendedData.SchemaData.SimpleData:
                #print(SimpleData)
                placemarkExtendedData_info.append(SimpleData)

        placemarkExtendedData_list.append(placemarkExtendedData_info)
    return placemarkExtendedData_list

# testing getPlacemarkExtendedData function
mykmlPlacemarkExtendedDataList = getPlacemarkExtendedData(mykmlPlacemarkList)
print(mykmlPlacemarkExtendedDataList)

In [ ]:
''' placemarkgpsDico function takes extendeddata input '''
''' then returns a gps dico from it'''
def placemarkGpsDico(extendeddata):
    dico = {"Time":str(extendeddata[7])+" "+str(extendeddata[8]),"Latitude":extendeddata[1],"Longitude":extendeddata[2],"Elevation":extendeddata[6]}
    return dico

# testing placemarkGpsDico
myextendeddatadico = placemarkGpsDico(mykmlPlacemarkExtendedDataList[0])
print(myextendeddatadico)

In [ ]:
def gpsPlacemarkList(placemarkExtendedDataList):
    gpsPlacemark_List = []
    for i in range(0,len(placemarkExtendedDataList)):
        gpsPlacemark_List.append(placemarkGpsDico(placemarkExtendedDataList[i]))
    return gpsPlacemark_List

myGpsPlacemarkList =  gpsPlacemarkList(mykmlPlacemarkExtendedDataList)
myGpsPlacemarkList

## KML functions helpers

In [ ]:
''' placeMarkNameList function takes a parsed kml file input'''
''' then returns the a list of Placemark names'''
def placemarkNameList(kmlroot):
    placemark_name = []
    for placemark in kmlroot.Document.Placemark:
        placemark_name.append(placemark.name)
    return placemark_name
    

''' getPlacemarkExtendedData function takes a placemarkname list input '''
''' then return'''
def getPlacemarkExtendedData(placemarkname):
    placemarkExtendedData_list = []
    for i in range(1,len(placemarkname)-1):
        
        placemarkExtendedData_info = []
        for placemark in root.Document.Placemark:
            if(placemark.name == placemarkname[i]):
                #print(placemark.name)
                temp = placemark
        for SimpleData in temp.ExtendedData.SchemaData.SimpleData:
                #print(SimpleData)
                placemarkExtendedData_info.append(SimpleData)

        placemarkExtendedData_list.append(placemarkExtendedData_info)
    return placemarkExtendedData_list


''' placemarkgpsDico function takes extendeddata input '''
''' then returns a gps dico from it'''
def placemarkGpsDico(extendeddata):
    dico = {"Time":str(extendeddata[7])+" "+str(extendeddata[8]),"Latitude":extendeddata[1],"Longitude":extendeddata[2],"Elevation":extendeddata[6]}
    return dico


''' gpsPlacemarkList function takes placemarkExtendedDataList input '''
''' then returng a list of gps dico of Placemark'''
def gpsPlacemarkList(placemarkExtendedDataList):
    gpsPlacemark_List = []
    for i in range(0,len(placemarkExtendedDataList)):
        gpsPlacemark_List.append(placemarkGpsDico(placemarkExtendedDataList[i]))
    return gpsPlacemark_List


''' gpsPlacemarkWorkflow function takes kmlroot document input '''
''' then return the gpsDicoList out from the Placemarks within kmlroot'''
def gpsPlacemarkWorkflow(kmlroot):
    placemarkNames = placemarkNameList(kmlroot)
    extendedDataList = getPlacemarkExtendedData(placemarkNames)
    gpsDicoList = gpsPlacemarkList(extendedDataList)
    return gpsDicoList

## Step by Step KML process

In [ ]:
from pykml import parser
root = parser.fromstring(open(kml_path4, 'r').read())

# get placemark name list
adour4PlacemarkName = placemarkNameList(root)
print(adour4PlacemarkName)

# get placemarks extended data
adour4ExtendedData = getPlacemarkExtendedData(adour4PlacemarkName)
print(adour4ExtendedData)

# get a gps dico from placemarks extended data
adour4GpsDicoList =  gpsPlacemarkList(adour4ExtendedData)
print(adour4GpsDicoList)

## Full KML process

In [ ]:
gps_path = '../kmlsamples'
kml_file4 = '19022020_Adour_4.kml'
kml_path4 = os.path.join(gps_path,kml_file4)
kml_file5 = '19022020_Adour_5.kml'
kml_path5 = os.path.join(gps_path,kml_file5)
kml_file6 = '19022020_Adour_6.kml'
kml_path6 = os.path.join(gps_path,kml_file6)

In [ ]:
# ADOUR4GPS
from pykml import parser
root = parser.fromstring(open(kml_path4, 'r').read())
adour4GPS = gpsPlacemarkWorkflow(root)
print(len(adour4GPS))
print(adour4GPS)

In [ ]:
# ADOUR5GPS
from pykml import parser
root = parser.fromstring(open(kml_path5, 'r').read())
adour5GPS = gpsPlacemarkWorkflow(root)
print(len(adour5GPS))
print(adour5GPS)

In [ ]:
# ADOUR6GPS
from pykml import parser
root = parser.fromstring(open(kml_path6, 'r').read())
adour6GPS = gpsPlacemarkWorkflow(root)
print(len(adour5GPS))
print(adour6GPS)

# Video Metadata

In [ ]:
! pip install opencv-python

In [ ]:
os.listdir('../kmlsamples/')

In [ ]:
import os

# video path
video_path = '../kmlsamples/'
video4_file = '19022020_Adour_4.mp4'
video5_file = '19022020_Adour_5.mp4'
video6_file = '19022020_Adour_6.mp4'
video4_path = os.path.join(video_path,video4_file)
video5_path = os.path.join(video_path,video5_file)
video6_path = os.path.join(video_path,video6_file)
print(video4_path)

In [ ]:
# Frame count using opencv
import cv2
cap = cv2.VideoCapture(video4_path)
length = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(int(length))

## Video info helper functions

In [ ]:
! brew install mediainfo

In [ ]:
# https://stackoverflow.com/questions/3844430/how-to-get-the-duration-of-a-video-in-python
import json
import subprocess

#===============================
def getMediaInfo(mediafile):
    cmd = "mediainfo --Output=JSON %s"%(mediafile)
    proc = subprocess.Popen(cmd, shell=True,stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, stderr = proc.communicate()
    data = json.loads(stdout)
    return data

#===============================
def getDuration(mediafile):
    data = getMediaInfo(mediafile)
    duration = float(data['media']['track'][0]['Duration'])
    return duration

#===============================
def getFrameRate(mediafile):
    data = getMediaInfo(mediafile)
    frameRate = float(data['media']['track'][0]['FrameRate'])
    return frameRate

#===============================
def getFrameCount(mediafile):
    data = getMediaInfo(mediafile)
    frameCount = float(data['media']['track'][0]['FrameCount'])
    return frameCount

## Video Adour4 GPS twisting

### Video4 infos

In [ ]:
# Video4 path
video4_file = '19022020_Adour_4.mp4'
video_path = '../kmlsamples/'
video4_path = os.path.join(video_path,video4_file)

# Get Video info
video4_metadata = getMediaInfo(video4_path)
video4_duration = getDuration(video4_path)
video4_frameRate = getFrameRate(video4_path)
video4_frameCount = getFrameCount(video4_path)

### Video4 Length seconds vs GPS points

In [ ]:
# Get info from Video metadata
print("############  Info about video ############")
# Checking duration in seconds and minutes
import datetime
duration = video4_duration
video4_second_minutes = str(datetime.timedelta(seconds=duration))
print("Video duration in second from metadata:",video4_duration)
print("Video duration in minutes from metadata:",video4_second_minutes)

# checking duration based on frameCount and frameRate
check_duration = video4_frameCount / video4_frameRate
print("Calculating video duration:",check_duration)
print("\n")


# Get info from Video metadata
print("############  Get GPS from KML video ############")

# ADOUR4GPS
from pykml import parser
root = parser.fromstring(open(kml_path4, 'r').read())
adour4GPS = gpsPlacemarkWorkflow(root)
print("Number of GPS points in KML file:",len(adour4GPS))

### OBSERVATION ###
'''
Video duration is 199 s for video4
GPS number of points is 156 - so there is less GPS point than the total duration of the video
There is "almost" a 1:1 relationship anyway between number of GPS point and duration in second
We assume that the system is approximatively taking a GPS point every second when possible
Because there are about 25 Frame per second, we assume that every 25 frame within same second has same gps coordinate
-> gpsHelper function: a function is required anyway to associate a GPS coordinate with every second of a video
-> when there is no GPS point at a given timestep= we can copy the previous one, or the next one or make an average
-> this is what will do the gpsHelper function
'''

In [ ]:
for gps in adour4GPS:
    print(gps)
# observation: 156 points GPS -> we want 200 GPS points because there is 199.986 s in video

In [ ]:
# createTime, createLatitude and createLongitude, create Elevation helper functions
import datetime
from datetime import datetime
from datetime import timedelta
def createTime(time):
    new_time = datetimeHelper(time)
    new_time = new_time + timedelta(seconds=1)
    new_time = datetime.strftime(new_time, '%d/%m/%Y %H:%M:%S')
    new_time = str(new_time)
    return new_time

def createLatitude(lat1,lat2):
    new_latitude = (lat1+lat2)/2
    new_latitude = round(new_latitude,6)
    return new_latitude

def createLongitude(long1,long2):
    new_longitude = (long1+long2)/2
    new_latitude = round(new_longitude,6)
    return new_longitude

def createElevation(elev1,elev2):
    new_elevation = (elev1+elev2)/2
    new_elevation = round(new_elevation,6)
    return new_elevation

# Testing helper functions
gps7 = adour4GPS[6]
gps8 = adour4GPS[7]
print(gps7)
print(gps8)

# Create Time, Latitude, Longitude, Elevation
print(gps7['Time'])
missing_time = createTime(gps7['Time'])
print(missing_time)
missing_latitude = createLatitude(gps7['Latitude'],gps8['Latitude'])
print(missing_latitude)
missing_longitude = createLongitude(gps7['Longitude'],gps8['Longitude'])
print(missing_longitude)
missing_elevation = createElevation(gps7['Elevation'],gps8['Elevation'])
print(missing_elevation)

# New GPS
new_gps = {'Time':missing_time,'Latitude':missing_latitude,'Longitude':missing_longitude,'Elevation':missing_elevation}
print(new_gps)

In [ ]:
gps7_time = datetimeHelper(gps7['Time'])
gps8_time = datetimeHelper(gps8['Time'])
gpsnew_time = datetimeHelper(new_gps['Time'])
delta = gpsnew_time - gps7_time
delta.total_seconds()

In [ ]:
duration = round(duration)
duration

In [ ]:
gps7_time = datetimeHelper(gps7['Time'])
gps8_time = datetimeHelper(gps8['Time'])
delta = int((gps8_time - gps7_time).total_seconds())

if delta > 1:
    missing_time = createTime(gps7['Time'])
    missing_latitude = createLatitude(gps7['Latitude'],gps8['Latitude'])
    missing_longitude = createLongitude(gps7['Longitude'],gps8['Longitude'])
    missing_elevation = createElevation(gps7['Elevation'],gps8['Elevation'])
    new_gps = {'Time':missing_time,'Latitude':missing_latitude,'Longitude':missing_longitude,'Elevation':missing_elevation}
print(new_gps)

In [100]:
temp_gps = [adour4GPS[0]]
i = 1
while i < 155: #up to 199
    delta = datetimeHelper(adour4GPS[i+1]['Time'])-datetimeHelper(adour4GPS[i]['Time'])
    delta = int(delta.total_seconds())
    if delta == 1:
        temp_gps.append(adour4GPS[i])
    else:
        missing_time = createTime(gps7['Time'])
        missing_latitude = createLatitude(gps7['Latitude'],gps8['Latitude'])
        missing_longitude = createLongitude(gps7['Longitude'],gps8['Longitude'])
        missing_elevation = createElevation(gps7['Elevation'],gps8['Elevation'])
        new_gps = {'Time':missing_time,'Latitude':missing_latitude,'Longitude':missing_longitude,'Elevation':missing_elevation}
        temp_gps.append(new_gps)
        temp_gps.append(adour4GPS[i+1])
    i = i+1
temp_gps.append(adour4GPS[155])

In [101]:
adour4GPS[155]

{'Time': '19/02/2020 15:02:39',
 'Latitude': 43.49635,
 'Longitude': -1.36943,
 'Elevation': 50.665527}

In [103]:
temp_gps[198]

{'Time': '19/02/2020 15:02:39',
 'Latitude': 43.49635,
 'Longitude': -1.36943,
 'Elevation': 50.665527}

### Adour4 GPS

## Datetime Helper for GPX/KML

In [ ]:
datetime_str = adour4GPS[0]['Time']
datetime_str

In [ ]:
# String Datetime to Datetime type
import datetime
from datetime import datetime

def datetimeHelper(datetime_str):
    '''
    Argument: datetime_str is a datetime string
    Output: datetimeHelper convert a datetime string with format %d/%m/%Y %H:%M:%S to format Y-m-d %H:%M:%S
    '''
    datetime_converted = datetime.strptime(datetime_str, '%d/%m/%Y %H:%M:%S')
    return datetime_converted


datetime_example = adour4GPS[0]['Time']
print(datetime_example)
print(datetimeHelper((datetime_example)))

In [ ]:
# KML vs GPX datetime
print("GPX datetime format")
print(mypoints[0]['Time'])
print("KML datetime format")
print(datetime_test)